## Download Feedback and Examples From a Test Project

When testing with Langsmith, all the traces, examples, and evaluation feedback are saved so you have a full audit of what happened.
This way you can see the aggregate metrics of the test run and compare on an example by example basis.

We will show how to export the feedback and examples from a Langsmith test project. The main steps are:

1. Create a dataset
2. Run testing
3. Export feedback and examples

## Setup

Install langchain and any other dependencies for your chain. We will install pandas as well for this walkthrough to put the retrieved data in a dataframe.

In [13]:
# %pip install -U langsmith langchain anthropic pandas --quiet

Note: you may need to restart the kernel to use updated packages.


In [18]:
import uuid
import os

unique_id = uuid.uuid4().hex[0:8]
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_ENDPOINT"] = ""
os.environ["LANGCHAIN_PROJECT"] = f"Retrieve feedback and examples {unique_id}"

## 1. Create a dataset

We will create a simple KV dataset with a poem topic and a constraint letter (which the model should not use).

In [19]:
from langsmith import Client

client = Client()

examples= [
    ("roses", "o"),
    ("vikings", "v"),
    ("planet earth", "e"),
    ("Sirens of Titan", "t"),
]

dataset_name = f"Download Feedback and Examples {unique_id}"
dataset = client.create_dataset(dataset_name)

for prompt, constraint in examples:
    client.create_example({"input": prompt, "constraint": constraint}, dataset_id=dataset.id, outputs={"constraint": constraint})

## 2. Run testing

We will use a simple custom evaluator that checks whether the prediction contains the constraint letter.

In [20]:
from typing import Any
from langchain.evaluation import StringEvaluator

class ConstraintEvaluator(StringEvaluator):
    
    @property
    def requires_reference(self):
        return True
    
    def _evaluate_strings(self, prediction: str, reference: str, **kwargs: Any) -> dict:
        # Reference in this case is the letter that should not be present
        return {
            "score": bool(reference not in prediction),
            "reasoning": f"prediction contains the letter {reference}",
        }

In [56]:
from langchain import chat_models, prompts, schema
from langchain.smith import RunEvalConfig

chain = (
    prompts.PromptTemplate.from_template("Write a poem about {input} without using the letter {constraint}. Respond directly with the poem with no explanation.")
    | chat_models.ChatAnthropic()
    | schema.output_parser.StrOutputParser()
)

eval_config = RunEvalConfig(
    custom_evaluators=[ConstraintEvaluator()],
    input_key="input",
)

test_results = client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=chain,
    evaluation=eval_config,
)

View the evaluation results for project 'test-dependable-wave-58' at:
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/8777c86e-9349-43bb-a471-c98ee2598774
[------------------------------------------------->] 4/4

## 3. Review the feedback and examples

If you want to directly use the results, you can easily access them in tabular format by calling `to_dataframe()` on the test_results. 

In [57]:
test_results.to_dataframe()

,ConstraintEvaluator,input,output,reference
38f8b732-79c3-45a2-b3b3-ff04d5a78c28,False,"{'input': 'Sirens of Titan', 'constraint': 't'}",Here is a poem about Sirens of Titan without ...,{'output': 't'}
82126087-4ae7-42b9-af51-a12ae0ee4dc3,False,"{'input': 'planet earth', 'constraint': 'e'}","Our home, our world, our earth\nSpinning thro...",{'output': 'e'}
1addd6f1-5e93-42f6-8753-b55eea5e3e8d,False,"{'input': 'vikings', 'constraint': 'v'}",Here is a poem about Vikings without using th...,{'output': 'v'}
376544fb-b475-4a84-86ea-5b28ea4a1aee,False,"{'input': 'roses', 'constraint': 'o'}",Here is a poem about roses without using the ...,{'output': 'o'}


If you want to fetch the feedback and examples for a historic test project, you can use the SDK:

In [58]:
# Can be any previous test projects
test_project = test_results['project_name']

In [70]:
import pandas as pd

runs = client.list_runs(project_name=test_project, execution_order=1)

df = pd.DataFrame(
    [
        {
            "example_id": r.reference_example_id,
            **r.inputs,
            **(r.outputs or {}),
            **{k: v for f in client.list_feedback(run_ids=[r.id]) for k, v in [(f"{f.key}.score", f.score), (f"{f.key}.comment", f.comment)]},
            "reference": client.read_example(r.reference_example_id).outputs
        }
        for r in runs
    ]
)
df

,example_id,input,constraint,output,ConstraintEvaluator.score,ConstraintEvaluator.comment,reference
0,376544fb-b475-4a84-86ea-5b28ea4a1aee,roses,o,Here is a poem about roses without using the ...,0.0,prediction contains the letter o,{'output': 'o'}
1,1addd6f1-5e93-42f6-8753-b55eea5e3e8d,vikings,v,Here is a poem about Vikings without using th...,0.0,prediction contains the letter v,{'output': 'v'}
2,82126087-4ae7-42b9-af51-a12ae0ee4dc3,planet earth,e,"Our home, our world, our earth\nSpinning thro...",0.0,prediction contains the letter e,{'output': 'e'}
3,38f8b732-79c3-45a2-b3b3-ff04d5a78c28,Sirens of Titan,t,Here is a poem about Sirens of Titan without ...,0.0,prediction contains the letter t,{'output': 't'}


## Conclusion

In this example we showed how to download feedback and examples from a test project. You can directly use the result object from the run or use the SDK to fetch the results and feedback. You can use this to further analyze and compare results.